In [ ]:
pip install flask


In [ ]:
pip install langchain


In [ ]:
pip install -U langchain-community


In [ ]:
pip install openai


In [ ]:
pip install faiss-cpu  # or faiss-gpu if you have GPU support


In [ ]:
pip install sentence-transformers


In [ ]:
pip install beautifulsoup4 requests


In [ ]:
pip install python-dotenv


In [ ]:
pip install flask langchain openai faiss-cpu sentence-transformers beautifulsoup4 requests python-dotenv


In [ ]:
pip install faiss-gpu


In [8]:
import os
import json
from flask import Flask, request, jsonify
from langchain.document_loaders import WebBaseLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.llms import OpenAI
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from langchain.schema import Document

# ✅ Securely load OpenAI API key (Avoid hardcoding in production!)
openai_api_key = "sk-proj-7zM7fxSz893IEeP481o7Kk2yEcY0MJnKqC9cg7wqT51wM930pdzDVILvChsgvl-CEk4CKLxapRT3BlbkFJDnt2jTZblgH5vBSAFMxq2lK6BwE0G4_K5uEupwrTxeHhSmjn_dKi8h0_nYsAA"

if not openai_api_key:
    raise ValueError("OpenAI API key is missing. Set it as an environment variable.")
else:
    print("✅ OpenAI API Key loaded successfully.")

# ✅ Step 1: Extract and Validate Data from Brainlox
URL = "https://brainlox.com/courses/category/technical"

try:
    loader = WebBaseLoader(URL)
    raw_documents = loader.load()

    if not raw_documents:
        raise ValueError("Failed to load data. The website might require JavaScript rendering.")

    print(f"✅ Loaded {len(raw_documents)} documents from {URL}")

    # Ensure documents are properly formatted
    documents = [
        Document(page_content=doc.page_content, metadata={"source": doc.metadata.get("source", URL)})
        for doc in raw_documents
    ]

except Exception as e:
    print(f"❌ Error loading documents: {e}")
    documents = []  # Ensure `documents` is always defined

# ✅ Validate Document Loading
if not documents:
    raise ValueError("No documents loaded. Check if the URL is accessible or requires different scraping methods.")


ModuleNotFoundError: Module langchain_community.document_loaders not found. Please install langchain-community to access this module. You can install it using `pip install -U langchain-community`

In [ ]:

# ✅ Step 3: Load FAISS Vector Store Securely (with dangerous deserialization allowed)
try:
    db = FAISS.load_local("faiss_index", embeddings, allow_dangerous_deserialization=True)
    retriever = db.as_retriever()
    print("✅ FAISS vector store loaded successfully.")
except Exception as e:
    print(f"❌ Error loading FAISS index: {e}")
    raise

# ✅ Step 4: Initialize LangChain Chat Model
try:
    llm = OpenAI(api_key=openai_api_key, temperature=0.5)
    memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

    chain = ConversationalRetrievalChain.from_llm(
        llm=llm,
        retriever=retriever,
        memory=memory
    )
except Exception as e:
    print(f"❌ Error initializing ConversationalRetrievalChain: {e}")
    chain = None

In [6]:

# deployment

from flask import Flask, request, jsonify

app = Flask(__name__)

@app.route("/chat", methods=["POST"])
def chatbot_response():
    # Your chatbot logic here
    if not chain:
        return jsonify({"error": "Chatbot is disabled. Set up an OpenAI API key or use a local LLM."}), 400
    return jsonify({"message": "Success!"})
    data = request.json
    user_input = data.get("message")

    if not user_input:
        return jsonify({"error": "Message is required"}), 400

    try:
        response = chain.run({"question": user_input})
        return jsonify({
            "response": response,
            "chat_history": memory.chat_memory.messages  # ✅ Returns conversation history
        })
    except Exception as e:
        return jsonify({"error": str(e)}), 500
if __name__ == "__main__":
    app.run(host="0.0.0.0", port=5000)





 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit
